# Extensions

The wrapper class functionalities can be extended using composition or inheritance.

## Composition

The concept of composition adds a component to a composite class. The relationship is such, that the composite class does not know the component class but wise versa. This allows to add additional functionality without changing the original code. Adding components to an existing wrapper class will be called "registering" here.

In [1]:
import h5rdmtoolbox as h5tbx
h5tbx.use('h5py')

2023-03-31_13:10:24,609 INFO     [__init__.py:63] Switched to convention "h5py"


## Built-in extensions
You may add your own extensions, but the packages comes already with some, that may be useful:

### 1. Vector
The `Vector` extension returns a `xr.Dataset` including the provided datasets when the `Vector` object is sliced. It just saves some time and makes it a one-liner to create the `xr.Dataset`.

In [2]:
from h5rdmtoolbox.extensions import vector

In [3]:
with h5tbx.H5File() as h5:
    h5.create_dataset('u', data=[1, 2, 3])
    h5.create_dataset('v', data=[2, 2, 2])
    v = h5.Vector('u', 'v')[1:3]
v

<xarray.Dataset>
Dimensions:  (dim_0: 2)
Dimensions without coordinates: dim_0
Data variables:
    u        (dim_0) int32 2 3
    v        (dim_0) int32 2 2

### 2. magnitude
This extension let's you compute the magnitude of given `xr.DataArray` objects, which goes well with the above extension:

In [4]:
from h5rdmtoolbox.extensions import magnitude # automatically available when vetor is imported

In [5]:
v.magnitude.compute_from('u', 'v')

<xarray.Dataset>
Dimensions:           (dim_0: 2)
Dimensions without coordinates: dim_0
Data variables:
    u                 (dim_0) int32 2 3
    v                 (dim_0) int32 2 2
    magnitude_of_u_v  (dim_0) float64 2.0 3.0

### Registration of standard attributes
During runtime, we can define so-called "standard attributes" and assign them to group or dataset classes. A more detailed explanation can be found [here](../conventions/standard_attributes.ipynb).

In the following example, the standard attribute shall write user information to attributes. For this the name and the orcid is expected. The new standard attribute shall be assigned to the `Dataset` class of the current wrapper:

In [6]:
from h5rdmtoolbox.conventions import StandardAttribute

In [13]:
class UserName(StandardAttribute):
    def set(self, username):
        # username shall be something like ['Max', '0000-1111-2222-3333']
        assert len(username) == 2
        super().set(name='user_name', value=user_name_data[0])
        super().set(name='user_orcid', value=user_name_data[1])
        
    def get(self):
        return super().get('user_name'), super().get('user_orcid')

Let's check if it worked out:

In [14]:
with h5tbx.File() as h5:
    ds = h5.create_dataset('test', shape=(2,))    
    ds.username = 'First User', '0000-0001-8729-0482'
    print(ds.username)
    try:
        ds.username = 'Second User'
    except ValueError as e :
        print(f'Could not add user due to: {e}')
    print(ds.username)
    h5.dump()

('First User', '0000-0001-8729-0482')
Second User


### Registration of datasets
We can also "register" dataset accessors. In the following example we add "device" as a "property with methods". So "device" seems to be a property which has a method "add". Such an implementation faclitates the interaction with HDF data, too. Note, that this "property-like" accessor is available for all `Dataset` objects from know on in this session:

In [15]:
from h5rdmtoolbox.wrapper.accessory import register_special_dataset
@register_special_dataset('device', h5tbx.File.Dataset(), overwrite=True)
class DeviceProperty:
    """Device Accessor class"""

    def __init__(self, ds):
        self._ds = ds
        self._device_name = 'NoDeviceName'
        
    def add(self, new_device_name):
        """adds the attribute device_name to the dataset"""
        self._ds.attrs['device_name'] = new_device_name
        
    @property
    def name(self):
        return self._ds.attrs['device_name']

In [16]:
with h5tbx.File() as h5:
    ds = h5.create_dataset('test', shape=(2,))
    print(type(ds))
    ds.device.add('my device')
    print(ds.device.name)

<class 'h5rdmtoolbox.wrapper.core.Dataset'>
my device


## Inheritance 

### New Wrapper Class

The main wrapper class around a HDF5 file in this package is `File` which uses the wrapper class `Group` for `h5py.Group` and `Dataset` for wrapping `h5py.Dataset`

In [17]:
import h5rdmtoolbox as h5tbx
import h5py

In [18]:
class MyDataset(h5tbx.wrapper.core.Dataset):
    def __repr__(self):
        return f'{self.name}\n{self.attrs}'
    
    @property
    def is_2d(self):
        """returns whether dataset is two-dimensional or not"""
        return self.ndim == 2

Next we create a group class with a special method, hat returns all datasets of that group. Also the `create_dataset` method is overwritten. Take care to return `MyDataset` at the end of the method, otherwise dataset class of the parent class is taken.

In [19]:
class MyGroup(h5tbx.wrapper.core.Group):
    """My group. It prints a logger-like info to the screen and 
    requires a user when creating datasets"""
    
    def get_all_datasets(self):
        """returns all datasets of this group"""
        return [k for k in self if isinstance(self[k], h5py.Dataset)]
    
    def create_group(self, name, *args, **kwargs):
        """overrite create_group method"""
        g = super().create_group(name, *args, **kwargs)
        print(f'creating a group named {g.name}')
        return MyGroup(g.id)
    
    def create_dataset(self, name, user, *args, **kwargs):
        ds = super().create_dataset(name, *args, **kwargs)
        print(f'creating a dataset named {ds.name}')
        ds.attrs.modify('user', user)
        return self._h5ds(ds.id)

The main file wrapper inherites from `File` ("root" parent was `h5py.File`) and the new group class. Next, we have set the group and dataset class again, since some methods in the file wrapper class will need that information when returning instances of those classes (e.g. dataset or group creation). Finally we define a new method which sets the user name to the root attributes:

In [20]:
class MyWrapper(h5tbx.wrapper.core.File, MyGroup):
    
    def set_user(self, user_name):
        self.attrs.modify('user', user_name)

register the dataset and group class in all classes. This is needed, so all return objects are of the newly defined types

In [21]:
MyGroup._h5ds = MyDataset
MyGroup._h5grp = MyGroup

MyDataset._h5ds = MyDataset
MyDataset._h5grp = MyGroup

In [22]:
h5 = MyWrapper()

In [23]:
h5.create_group('dwa')

creating a group named /dwa


<HDF5 wrapper group "/dwa" (members: 0, convention: "None")>

In [24]:
g = h5.create_group('grp', overwrite=True)
gg = g.create_group('grp', overwrite=True)

creating a group named /grp
creating a group named /grp/grp


In [25]:
ds = gg.create_dataset('hello', shape=(2,3), user='Max', overwrite=True)

creating a dataset named /grp/grp/hello


In [26]:
h5.set_user('new_user')

In [27]:
h5

<HDF5 (convention: "None") file "tmp3.hdf" (mode r+)>